In [27]:
import pandas as pd
import re
import pandas as pd 
# import transformers
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import numpy as np
punctuation = '!"#$%&\'()*+,-./:;<=>?@\\^_`{|}~“”'


### Process data

In [2]:
df = pd.read_csv("/scratch/venia/web2wiki/data/web_content/iterative_coding_sample/cleaner_data.csv")


In [3]:
df = df[df["link_type"] == 1]

### Clean text

In [4]:
def preprocess_text(text):
    text = str(text)
    text = re.sub(r'http\S+', '', text)
    text = text.lower()  # Lowercase text
    text = re.sub(f"[{re.escape(punctuation)}] ", "", text)  # Remove punctuation
    text = " ".join(text.split())  # Remove extra spaces, tabs, and new lines
    return text

def mask_tokens(text):
    tokens = text.split(" [break] ")
    tokens.pop(1)
    masked = " [MASK] ".join(tokens)
    return masked



def no_masks(text):
    tokens = text.split(" [break] ")
    tokens.pop(1)
    masked = " ".join(tokens)
    masked = re.sub(f"break", "", masked)  # Remove punctuation
    return masked
from langdetect import detect

def get_lang(x):
    x = str(x)
    try: 
        return detect(x)
    except:
        return "chinese"


In [5]:
df["clean_text"]=df["nbhd_text"].apply(lambda x: preprocess_text(str(x)))
dff = df[df["clean_text"].apply(lambda x: x.split(" [break] ")).map(len)>1]
dff["masked_text"] = dff[["clean_text","order"]].apply(lambda x: no_masks(x["clean_text"]),axis = 1)
print(len(dff))

651279


/tmp/ipykernel_807373/1336889250.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff["masked_text"] = dff[["clean_text","order"]].apply(lambda x: no_masks(x["clean_text"]),axis = 1)


In [6]:
dff = dff.sample(len(dff))

In [7]:
# dff = dff[dff["order"] == 1] 

In [8]:
dff =dff.drop_duplicates(["domain" ])

In [9]:
df2 = dff.copy()

In [10]:
# dff = df2

In [11]:
predict = "order"
x_vars = "masked_text"

x=dff[x_vars]
y = dff[predict]

In [12]:
from nltk.corpus import words

In [13]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to /home/veselovs/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [14]:
voc = words.words() 
voc = voc + ["Wikipedia", "wiki","pedia"]

In [15]:
voc = list(set(voc))

In [16]:
dff = dff[dff["masked_text"].apply(lambda x: len(str(x).split(" ")) > 1)]

In [17]:
zero = dff[dff["order"] == 0]

In [18]:
one = dff[dff["order"] == 1]

In [81]:
cv = CountVectorizer( max_features=10000, ngram_range = (1,1), stop_words = "english")
x_cv = cv.fit_transform(dff[x_vars].astype(str))

In [82]:
cv_df = pd.DataFrame(x_cv.toarray(), columns = cv.get_feature_names(), index=dff.index)

/home/veselovs/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [83]:
cv_df["order"] = dff["order"]

In [94]:
sampled = cv_df.groupby("order").sample(1000)

In [95]:
order_count = sampled.groupby("order").sum()

In [96]:
probs = order_count.divide(order_count.sum(axis=1),axis=0).T

In [97]:
from scipy.stats import entropy

In [98]:
entropy(probs[0])

7.890499565900473

In [99]:
entropy(probs[1])

8.069159082309247

In [100]:
entropy(probs[2])

7.76162656851199

In [23]:
tfidf = TfidfVectorizer( max_features=5000, ngram_range = (1,1), vocabulary = voc, stop_words = "english")
x_tfidf =tfidf.fit_transform(dff[x_vars].astype(str))

In [93]:
tf = x_tfidf.toarray()

In [99]:
tfidf = TfidfVectorizer( max_features=20000, ngram_range = (1,1), vocabulary = voc)
zero_tfidf =tfidf.fit_transform(zero[x_vars].astype(str))

/home/veselovs/miniconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:1322: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


/home/veselovs/miniconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:1322: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [21]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(x_tfidf,dff["order"])

In [22]:
from sklearn.linear_model import LogisticRegression

In [109]:
one["link_count"] = one.groupby("wiki_links")["url"].transform("count")

/tmp/ipykernel_3516391/2880218716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one["link_count"] = one.groupby("wiki_links")["url"].transform("count")


In [145]:
one_[one_["masked_text"].str.contains("twitter")].iloc[1]

url                 http://www.hackmageddon.com/2011/06/02/seeds-o...
wiki_links          https://bufferapp.com/add?url=https://www.hack...
is_tag_footer                                                     0.0
is_tag_header                                                     0.0
is_tag_aside                                                      0.0
is_tag_sup                                                        0.0
is_tag_cite                                                       0.0
is_tag_p                                                          0.0
is_tag_article                                                    1.0
is_class_footer                                                   0.0
is_class_header                                                   0.0
is_class_sidebar                                                  0.0
is_class_comment                                                  0.0
previous_header                                                  None
nbhd_text           

In [116]:
one_ = one[one["link_count"]<20]

In [121]:
tfidf = TfidfVectorizer( max_features=5000, ngram_range = (1,1))
one_tfidf =tfidf.fit_transform(one_[x_vars].astype(str))

In [122]:
lr_one = LogisticRegression(multi_class='multinomial', solver='lbfgs', C=0.3).fit(one_tfidf,one_["link_type"].astype(str))

In [101]:
lr_zero = LogisticRegression(multi_class='multinomial', solver='lbfgs', C=0.3).fit(zero_tfidf,zero["link_type"].astype(str))

In [25]:
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', C=0.5).fit(x_tfidf,dff["order"].astype(str))

/home/veselovs/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [94]:
lr2 = LogisticRegression(multi_class='multinomial', solver='lbfgs', C= 0.3).fit(x_tfidf,dff["link_type"].astype(str))

In [123]:
import eli5
eli5.show_weights(estimator=lr_one, 
                  feature_names= list(tfidf.get_feature_names()),
                 top=(10, 10))

/home/veselovs/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+0.981,performer
+0.892,expand
+0.840,imdb
+0.795,is
+0.772,name
+0.764,defined
+0.749,television
+0.732,ft
+0.727,include
+0.726,spanish


In [26]:
import eli5
eli5.show_weights(estimator=lr, 
                  feature_names= list(tfidf.get_feature_names()),
                 top=(20, 20))

/home/veselovs/miniconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
2022-10-22 16:58:51.542634: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-22 16:58:51.542685: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/veselovs/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [95]:
import eli5
eli5.show_weights(estimator=lr2, 
                  feature_names= list(tfidf.get_feature_names()),
                 top=(50, 50))

/home/veselovs/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+1.204,social
+1.038,confirm
+1.035,performer
+0.886,attribution
+0.864,expand
+0.829,defined
+0.806,is
+0.792,sometimes
+0.772,style
+0.769,include


In [83]:
import eli5
eli5.show_weights(estimator=lr, 
                  feature_names= list(tfidf.get_feature_names()),
                 top=(50, 50))

/home/veselovs/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [68]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(x_train, y_train)

In [70]:
y_hat = clf.predict(x_test)

In [74]:
from sklearn.metrics import confusion_matrix

In [76]:
confusion_matrix(y_test,y_hat)

array([[  37,  387],
       [  34, 1158]])

In [73]:
dff

,url,wiki_links,is_tag_footer,is_tag_header,is_tag_aside,is_tag_sup,is_tag_cite,is_tag_p,is_tag_article,is_class_footer,...,evidence,attribution,2nd_order,order,first_class,link_type,domain,domain_count,clean_text,masked_text
546198,http://blackbutterfly7.wordpress.com/tag/settl...,http://en.wikipedia.org/wiki/Honolulu,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0,0.0,1,ws,1,blackbutterfly7.wordpress.com,7,"dollars (about 71,000 euros)the women were arr...","dollars (about 71,000 euros)the women were arr..."
984495,http://flintservice.org/california/sun-city/co...,https://en.wikipedia.org/wiki/Chamfer,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0,0.0,1,ws,1,flintservice.org,5228,or by using powered floatsafter the concrete h...,or by using powered floatsafter the concrete h...
508011,http://charlesasullivan.com/15321/the-mind-and...,https://en.wikipedia.org/wiki/David_Hume,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0,0.0,1,ws,1,charlesasullivan.com,16,onger list of names such as the philosopher/th...,onger list of names such as the philosopher/th...
212753,http://wikien4.appspot.com/wiki/Dutch_art,http://en.wikipedia.org/w/index.php?title=Liec...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,1,ws,1,wikien4.appspot.com,6348,awbania andorra armenia azerbaijan bewarus bos...,awbania andorra armenia azerbaijan bewarus bos...
177061,http://pungenerator.org/puns?q=puppy,https://en.wikipedia.org/wiki/Bloody Buccaneers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,1,ws,1,pungenerator.org,9393,puppy buccaneers tweet [break] bloody buccanee...,puppy buccaneers tweet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201444,http://www.usmilitariacollection.com/boutique/...,https://en.wikipedia.org/wiki/Philip_Morris_In...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0,0.0,0,NaN,1,www.usmilitariacollection.com,2,"es paquets philip morris,… l’astuce consiste à...","es paquets philip morris,… l’astuce consiste à..."
164702,http://www.intel.com/content/www/us/en/program...,http://en.wikipedia.org/wiki/Floor_and_ceiling...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,1,ws,1,www.intel.com,2,related information [break] floor and ceiling ...,related information pci local bus specificatio...
906039,http://www.phentermineonline.com/blog/looking-...,https://en.wikipedia.org/wiki/Jeremy_Kyle,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0,0.0,1,ws,1,www.phentermineonline.com,2,l soul mate too they say all is fair in love a...,l soul mate too they say all is fair in love a...
641069,http://blog.bricsys.com/bricscad-bim-geometry-...,https://en.wikipedia.org/wiki/Bruges,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0,0.0,1,ws,1,blog.bricsys.com,2,geometry first? i was wandering the streets of...,geometry first? i was wandering the streets of...
